# Pre-Pre-Processing

Before I start performing any EDA or modeling, I will pre-emptively split my data to have a separate training set of data.

Since the data consists of temporal data (hotel reservations across multiple years), I will split the data into two groups based on two datetime features: Booking Date and the Arrival date.

---

# Consulting ChatGPT

I asked ChatGPT to help me decide if one feature would be better than the other. It's responses are as follows:

## Booking Date

### Pros:
> * *Forecasting Lead Time:* Splitting by booking date allows you to model the lead time between booking and arrival, which is valuable for forecasting future demand well in advance.
> * *Real-world Scenario Modeling:* This approach mirrors real-world scenarios more closely, where predictions are made as bookings occur, regardless of the future arrival date.
> * *Dynamic Pricing and Revenue Management:* Particularly useful for applications in dynamic pricing and revenue management, where understanding booking patterns ahead of time can inform strategic decisions.

### Cons:
> * *Increased Complexity:* The time gap between booking and arrival dates introduces additional complexity, as you need to account for changes over time that could affect the stay (e.g., cancellations, modifications).



## Arrival Date

### Pros:
> * *Cleaner Data Relationships:* Utilizing the arrival date might result in cleaner, more straightforward modeling, as it directly ties the prediction to the period of service or product consumption.
> * *Seasonality and Trend Analysis:* Easier to model seasonal trends and effects, as the arrival date directly corresponds to the period being analyzed.
> * *Accuracy in Performance Metrics:* Predictions based on arrival date can be more closely aligned with actual occupancy and revenue, potentially improving model accuracy in terms of performance metrics.

### Cons:

> * *Reduced Forecasting Horizon:* The model may be less effective at predicting bookings well in advance since it's oriented around the arrival date. This could limit its usefulness for long-term planning.
> * *Possible Lag in Actionable Insights:* May not provide as much lead time for implementing strategies based on the predictions, such as staffing or promotional offers, since the focus is on the period closer to the actual stay.

---

# Consultation Conclusions

After considering ChatGPT's suggestions and insights, I will take a more greedy approach and create separate datasets for both the booking and arrival dates. This will give me more flexibility when modeling as I will have different time perspectives to utilize for different purposes (e.g., future forecasting vs. analyzing actualized performance).

---

# Date Preparation

Before I can split the datasets, I need to perform some slight feature engineering. The source datasets do not have an exact datetime feature for the arrival date, only for the booking date. I will use the separate Year, Month, and Day of Month features to create an `Arrival_Date` feature, then use this feature for splitting my data.

In [1]:
import pandas as pd

In [2]:
## Sharing 'df_data' to reuse code from a prior notebook
path1 = './data/H1.parquet'
df_data = pd.read_parquet(path1)

# ## Sharing 'df_data' to reuse code from a prior notebook
# path2 = './data/H2.parquet'
# df_data = pd.read_parquet(path2)

In [ ]:
## Convert Arrival columns to strings

arrival_date_cols = ['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth']

arrival_date_cols_str = df_data[arrival_date_cols].astype(str)
arrival_date_cols_str.head()

In [13]:
import pandas as pd

# Define file paths
csv_file_paths = ['C:\\Users\\Ben\\Downloads\\H1.csv', 'C:\\Users\\Ben\\Downloads\\H2.csv']
parquet_file_paths = ['C:\\Users\\Ben\\Downloads\\H1.parquet', 'C:\\Users\\Ben\\Downloads\\H2.parquet']

# Convert each CSV to Parquet with brotli compression
for csv_path, parquet_path in zip(csv_file_paths, parquet_file_paths):
    df = pd.read_csv(csv_path)
    df.to_parquet(parquet_path, engine='pyarrow', compression='brotli')
    print(f'Converted {csv_path} to {parquet_path} with brotli compression.')

Converted C:\Users\Ben\Downloads\H1.csv to C:\Users\Ben\Downloads\H1.parquet with brotli compression.
Converted C:\Users\Ben\Downloads\H2.csv to C:\Users\Ben\Downloads\H2.parquet with brotli compression.
